# Foundations of Data Science
## S1 Week 05: Data wrangling II - groupby and regular expressions

**Learning outcomes:** 
In this lab, we will build on what you have previously learned on preparing data for future use. By the end of the lab you should be able to:

- use regular expressions to parse textual data
- organise columns into indicator variables when appropriate
- apply group-wise computations to your data

In this week's lab, you will be analysing data from the course survey from last year, as well as data obtained from the department of Informatics timetabling. There are many instances whereby you might want to create a form and analyse user responses to it - let this be a lesson in how (not?) to design a form 🤔.

**Data description:** The data from the first dataset for this lab originates from last year's course optional survey, which included several questions given to past students inquiring about their remote working setup, Python expertise and interests. The data was anonymised prior to being provided for this lab. The second dataset is the Informatics timetables which can be obtained from the web.

In [2]:
import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
import time

## A. Exploring the survey data

### A.1 Loading and preparing the survey data

**Exercise 01:** Load `survey_data.xlsx` into a variable `survey_data`.

In [5]:
survey_data = pd.read_excel("../FDS-S1-05-data-wrangling-02/datasets/survey_data.xlsx")

Whenever working with data, remember it's important to first get an idea of what your data looks like. Let's start by having a look at the dtypes of the columns.

In [6]:
survey_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 6 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   ID                                                                                 331 non-null    int64 
 1   My  preference for tutorial location is                                            329 non-null    object
 2   When the time is 10:00 (10am) in the UK, what is the time in my current location?  325 non-null    object
 3   My expertise with Python is                                                        330 non-null    object
 4   My IT equipment includes                                                           330 non-null    object
 5   My interests include                                                               325 non-null    object
dtypes: 

With the exception of 'ID', pandas has inferred the types of the remaining columns as Python 'objects' (text or mixed numeric and non-numeric values). In particular, notice that we have a column named as though it should contain a timestamp, '...what is the time in my current location?'. We'll look at converting this to a proper time format.

But first, you might (not!) have noticed the double space typo in the second column name. Let's tidy up some of the names so they're easier to reference later. Using [`pandas.DataFrame.rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.htmlhttps://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html), we can specify the names of existing columns and what they should be renamed to.

In [7]:
survey_data = survey_data.rename(columns={survey_data.columns[1]: 'Preferred tutorial loc', # This is equivalent to {'My  preference for tutorial location is': 'Preferred tutorial loc', ...
                                          survey_data.columns[2]: 'Time at 10:00 BST',
                                          survey_data.columns[3]: 'Python expertise',
                                          survey_data.columns[4]: 'IT equipment',
                                          survey_data.columns[5]: 'Interests'})
print("Column names:", list(survey_data.columns))

Column names: ['ID', 'Preferred tutorial loc', 'Time at 10:00 BST', 'Python expertise', 'IT equipment', 'Interests']


Although date and time are not a data type of their own in Python, the [Datetime](https://docs.python.org/3/library/datetime.html#module-datetime) module supplies classes and functions to work with them. Representing textual time columns as Datetime objects can be handy for querying rows that meet specific time conditions and performing simple date and time arithmetic. The pandas equivalent to the Python Datetime object is [Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html).

### A.2 Converting textual time data to a datetime representation

One option is to use [`pd.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to convert the string values in a column to a Timestamp if the string meets a format of our choice (specified using [strftime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) date format codes, e.g. `%m/%d/%Y` to match with `12/19/1995`). This function has the pandas `errors` argument that we've seen before in lab 2 (in `pd.to_numeric`), allowing us to `raise` an error if a value doesn't match the desired format, `coerce` it to be set as `NaT` or `ignore` it to return it as it is.

If the values in our time column don't follow a rigid string format already, we might end up losing a lot of our data.

**Exercise 02:** Print out the _unique_ values in the 'Time at 10:00 BST' column and their corresponding count in the column.

**Hint:** A single pandas method can be used for this.

In [8]:
pd.unique(survey_data['Time at 10:00 BST'])

array(['17:00', '10:00', '12:00', '15:00', '18:00', '10am', '14:00',
       '11:00', '5pm', '13:00', '10 am', '14:30', '12:54', '16:00',
       '1300 (when 10:00BST) 1400(if 10:00GMT)', '11:00 AM', nan, '1000',
       '10:00AM', '10', '10:00am', '19:00', '10.00', '17：00', '18',
       '10:00 am', '14:54', '1700', '10:00(10am)', '18：00', '02:00'],
      dtype=object)

**Discussion 01:** Did you expect to see so many variations in the format of the values in the time column? Discuss with your partner the implications of this on using the function [`pd.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to convert the column to a datetime representation. What [`strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) format could you use to successfully convert most of the values?

Your answer:

How else could we clean up more of the textual time column prior to attempting to convert it a useful datetime type?

### Regular expressions

You may have encountered [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) before, which are also referred to as __regexs__. These are sets of character combinations that Python (and many other programming languages) can understand and use as rules to search through character strings.

You can use these regex rules to ask questions such as “Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”. You can also use regexs to modify or split apart a string in various ways.

We can use the Python [`re`](https://docs.python.org/3/library/re.html) package for this. Some simple examples are shown in the cell below.

In [9]:
# Simple example: a pattern that matches with the first name Sarah or Sara
pattern = '^Sara(h?)$'
# ^ matches with the start
# () contains an expression to capture and group
# ? zero or one occurrence
# $ matches with the end

# re.search looks for any location where the regex matches; re.match checks only the beginning.
# If successful, it returns a Match object with info such as where the match starts and ends,
# the substring it matched, and more. If no match is found, None is returned.
print(re.search(pattern, 'Sarah'))
print(re.search(pattern, 'Sara'))
print(re.search(pattern, 'sarah'))

# re.findall returns all substrings where the regex matches as a list
eg_sentence = 'Sarah went to the park. Sarah wished it would rain.'
print(re.findall('Sara(h?)', eg_sentence))
print(re.findall('Sara(?:h)', eg_sentence))
# The last statement uses (?:...), a non-capturing version of regular parentheses.
# It matches whatever regex is inside the parentheses, but the substring matched by the group cannot
# be retrieved after performing a match or referenced later in the pattern (re.findall otherwise returns the matched group only).

<re.Match object; span=(0, 5), match='Sarah'>
<re.Match object; span=(0, 4), match='Sara'>
None
['h', 'h']
['Sarah', 'Sarah']


Several [methods](https://docs.python.org/3/library/re.html#match-objects) can be called on the 'Match' object that's returned to us. One useful method is `group()` which returns one or more string subgroups of the match. By default it returns the entire match, but if we pass an integer (up to 100) the substring matching the corresponding parenthesised group is returned.

**Excerise 03:** Can you consult the Python [regex syntax](https://docs.python.org/3/library/re.html) documentation to construct a general regular expression pattern to extract the string 'bulldog' from the string 'data/CIFAR-10/images/bulldog_146.jpg'.

In [14]:
search_str = 'data/CIFAR-10/images/bulldog_146.png'
print(re.findall('bulldog', search_str))

['bulldog']


So, back to cleaning up the time data column. We can use regular expressions to define a set of rules that progressively capture the variations in the format of strings in the 'Time at 10:00 BST' column, and substitute the matched sub-strings appropriately to convert them to a consistent string format.

**Exercise 04:** In the cell below, `time_clean()` is a stub of a function that cleans the times. We've done some of the more tricky replacements for you, and your task is to try making some of the other replacements. Because this could be a very time-consuming and frustrating exercise, we've given the solution in the file `functions.py`. If you want to move on and come back to the exercise, you can do so by uncommenting the `import` line below.

In [27]:
def time_clean(t):
    """Clean a string from the survey data to a consistent string format ('%H:%M').
    This function should handle the string cases found in Exercise 02."""
    if isinstance(t, str):  # NaN values will be of type float
        # re.sub replaces the parts of a string matching a pattern with a desired replacement
        t = re.sub('：', ':', t) # Replace "fullwidth colon" (Unicode character U+FF1A) with standard colon
        t = re.sub(' +', '', t) # Remove spaces in the middle of the string
        if re.match('^([^:]+)[Pp][Mm]', t): # 5pm -> 17:00
            hour = re.sub('^([^:]+)[Pp][Mm]', '\\1', t)
            t = str((int(hour) + 12)) + ":00"
        # Your answer:    
        # More lines in the format
        
    
    return(t)

# Uncomment the following for a function that works
from solutions import time_clean 

# Example string to clean
example_time = "10:00(10am) "
print(example_time, "->", time_clean(example_time))

10:00(10am)  -> 10:00


**Exercise 05:** Use `time_clean()` to convert each value in the 'Time at 10:00 BST' column to a pandas Timestamp format.

In [20]:
survey_data["Time at 10:00 BST"] = survey_data["Time at 10:00 BST"].apply(time_clean)

Now that our column is represented as a datetime, we can perform time-related queries on it such as,

In [21]:
after_midday = survey_data['Time at 10:00 BST'] >= "12:00"
pm_df = survey_data.loc[after_midday]
print("Number of users in the 'pm' during the lecture:", pm_df.shape[0])

Number of users in the 'pm' during the lecture: 82


**Exercise 06:** Query the data to determine:

a) the earliest time the lecture takes place for any of the students.

b) the number of users that are local to UK time.

In [64]:
survey_data["Time at 10:00 BST"] = pd.to_datetime(survey_data["Time at 10:00 BST"], format="%H:%M")
survey_data["Time at 10:00 BST"].min()

Timestamp('1900-01-01 02:00:00')

### A.3 Dummy / Indicator variables

The survey data has several columns that contain categorical entries, and users can have multiple categorical options for 'IT equipment' and 'Interests' which are currently separated by semicolons. 

Depending on the analysis you might want to perform, it can be beneficial and neccessary to create dummy or indicator variables in order to handle numerical computations on your data. These are binary variables that indicates whether a separate categorical variable takes on a specific value for an entry.

[`pd.Series.str.get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html) is a function we can use to convert our categorical variable into dummy/indicator variables. We use dummy variables when applying regression and classification algorithms to categorical data. It also has a handy argument for specifying a separator on which to split our multiple entries by. After converting to this format, the presence of the category receives a value of 1 and in its absence the value is 0.

In [54]:
equipment_expanded = survey_data['IT equipment'].str.get_dummies(sep=";")
equipment_expanded.head()

,Internet is mostly high speed and reliable but there are often power outages,Average internet,Camera,Desktop,"High speed internet, but it may break for two or three minute several times a day",High speed reliable internet,Laptop,Mobile phone,Not sure about internet speed as not moved into flat but according to the letting agency they have never had complaints about it,Tablet,wifi that not reliable
0,0,0,0,0,0,1,1,1,0,1,0
1,0,0,0,1,0,0,1,1,0,0,0
2,0,0,1,1,0,1,1,1,0,1,0
3,0,0,1,0,0,0,1,1,0,0,0
4,0,0,1,0,0,0,1,1,0,0,0


In [65]:
survey_equip_expanded = pd.concat([survey_data, equipment_expanded], axis=1)
survey_equip_expanded.head()

,ID,Preferred tutorial loc,Time at 10:00 BST,Python expertise,IT equipment,Interests,Internet is mostly high speed and reliable but there are often power outages,Average internet,Camera,Desktop,"High speed internet, but it may break for two or three minute several times a day",High speed reliable internet,Laptop,Mobile phone,Not sure about internet speed as not moved into flat but according to the letting agency they have never had complaints about it,Tablet,wifi that not reliable
0,1,Online,1900-01-01 17:00:00,Some experience,Laptop;High speed reliable internet;Tablet;Mob...,Science;Nature;Politics;Music;Sports;,0,0,0,0,0,1,1,1,0,1,0
1,2,Online,1900-01-01 17:00:00,No experience,Laptop;Mobile phone;Desktop;,Science;Sports;,0,0,0,1,0,0,1,1,0,0,0
2,3,Online,1900-01-01 17:00:00,Some experience,Laptop;Desktop;Tablet;Camera;Mobile phone;High...,Science;,0,0,1,1,0,1,1,1,0,1,0
3,4,Online,1900-01-01 10:00:00,No experience,Laptop;Mobile phone;Camera;,Politics;Sports;Science;,0,0,1,0,0,0,1,1,0,0,0
4,5,On campus,1900-01-01 10:00:00,No experience,Laptop;Camera;Mobile phone;,Science;Sports;Politics;,0,0,1,0,0,0,1,1,0,0,0


**Exercise 07 (optional):** You will see that some of the column headers are very long. This is because, along with a categorical list of 'IT equipment', users could also specify free-written responses to contain extra details. Write a regular expression that removes any sub-entry specified which is longer than three words and apply it to the 'IT equipment' column. Make sure to keep an eye out for NaN values and whitespace. Then repeat the conversion above to dummy variables.

In [ ]:
# Your code

## B. School of Informatics timetabling

### B.1 Loading the Informatics timetables

The timetables for courses in the School of Informatics can be found online (e.g. see ours [here](https://www.ted.is.ed.ac.uk/UOE2021_SWS/TIMETABLE.ASP?OBJECTCLASS=MODULE&IDENTIFIER=INFR08030_SS1_YR&STYLE=TEXTSPREADSHEET&TEMPLATE=SWSCUST+OBJECT+TEXTSPREADSHEET&WEEK=9-37)). We downloaded the webpage specifying the timetables for all of the courses within the School of Informatics for Semester 1 in 2021-22; see `SWSCUST Object TextSpreadsheet.html` in the `datasets` folder. The timetables are written in Hypertext Markup Language (HTML), a language which defines the meaning and structure of web content. It's exactly what you would see if you looked at the page source in the browser.

We will learn more about the details of web-scraping in a later lab. Today we will focus on using pandas to read a HTML file and performing group-wise computations across the data.

One option for specifically parsing HTML tables is to use [`pd.read_html()`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html). This function searches for the `<table>` elements in the HTML page and returns them as a list. 

We advise opening 'SWSCUST Object TextSpreadsheet.html' in your web-browser to have a better look at it. You might notice that many of the tables are empty, under certain days of the week for example. We can use the `match` argument to only return tables that contain text matching a string or regular expression that we specify. The value defaults to `.+` (i.e. matches any non-empty string).

In [3]:
timetable_data_path = os.path.join(os.getcwd(), 'datasets', 'SWSCUST Object TextSpreadsheet.html')
timetable_data_list = pd.read_html(timetable_data_path, match='Activity', header=0)
print("Tables parsed:", len(timetable_data_list))
print("Example table:")
timetable_data_list[2].head()

Tables parsed: 160
Example table:


,Activity,Description,Type,Start,End,Weeks,Building,Room,Staff
0,Accelerated Natural Language Processing - Tuto...,NaN,*Tutorial,10:00,10:50,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",Appleton Tower,AT_G.05,NaN
1,Accelerated Natural Language Processing - Drop...,NaN,*Seminar - Online Live,12:10,13:00,Sem1 wk1-Sem1 wk11,NaN,NaN,NaN
2,Accelerated Natural Language Processing - Tuto...,NaN,*Tutorial,13:10,14:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,NaN
3,Accelerated Natural Language Processing - Tuto...,NaN,*Tutorial,14:10,15:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,NaN
4,Accelerated Natural Language Processing - Tuto...,NaN,*Tutorial,15:10,16:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,NaN


**Exercise 08:** We currently have a list of DataFrames, where each DataFrame is a table parsed from the HTML page. Can you create one DataFrame that contains all of these tables? Assign it the variable name `timetable_data`.

- **Hint 1:** Printing the shape of the resulting DataFrame should return (501, 9).
- **Hint 2:** You met the required function in the first Data Wranging lab, but there its argument was a list with only two members.

In [29]:
timetable_data = pd.concat(timetable_data_list)

In [7]:
timetable_data.shape

(501, 9)

Regular expressions can also be useful for stripping extra whitespace or unnecessary characters from your data.

**Exercise 09:** Use a regular expression to strip the '*' characters from the values in the 'Type' column.

In [31]:
timetable_data['Type'].apply(lambda x: re.sub('\*', '', x))

0    Computer Workshop - Online Live
0              Lecture - Online Live
0                           Tutorial
1              Seminar - Online Live
2                           Tutorial
                  ...               
1           Laboratory - Online Live
0              Lecture - Online Live
0                          Practical
0                            Lecture
0                            Lecture
Name: Type, Length: 501, dtype: object

Since `pd.read_html` only parses `<table>` elements, with the layout of 'SWSCUST Object TextSpreadsheet.html' its tricky to neatly associate each table with its associated day of the week (which are specified in `<p>` tags) and course. HTML can be notoriously messy to parse. Thankfully, there are several external libraries which can be useful for this task, some of which we'll look at in a later lab.

The current 'Activity' column of our data contains the name of the course, and sometimes specific details regarding the activity. For now, we'll try to remove the latter details so we can later group our timetables by course.

In [32]:
def course_name_from_activity(activity_str):
    """Attempts to extract the course name from the 'Activity' column
    of the timetable data. Warning: this function is not perfect! There are still some
    exceptions in the formatting of the course names which are not captured."""
    if re.search('[-]', activity_str):
        match = re.match('^(Informatics \d - )?.*(?= - )', activity_str)
        assert match is not None, f"No match found for '{activity_str}'."
        return match.group()  # Return the matched string (instead of Match object)
    return activity_str

timetable_data["Course Name"] = timetable_data["Activity"].apply(lambda x: course_name_from_activity(x))

num_activities = timetable_data["Activity"].nunique()
extracted_courses = timetable_data["Course Name"].nunique()
print(f"{num_activities} detailed activities reduced to {extracted_courses}.")

402 detailed activities reduced to 56.


### B.2 Groupwise computations

By _groupby_ (or _split-apply-combine_) we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria.
* Applying a function to each group independently. This could include computing a summary statistic for each group, such as a mean or count; performing some group-specific computation such as standardising the data within a group; as well as filtering to discard data that has only a few members, for example.
* Combining the results into a data structure.

In [33]:
# First let's drop some of the empty or redundant columns
timetable_data = timetable_data.drop(['Activity', 'Description', 'Staff'], axis=1)
timetable_data.head()

,Type,Start,End,Weeks,Building,Room,Course Name
0,Computer Workshop - Online Live,14:10,15:00,"Sem1 wk2-Sem1 wk3, Sem1 wk7, Sem1 wk9",NaN,NaN,Accelerated Natural Language Processing
0,*Lecture - Online Live,11:10,12:00,Sem1 wk1-Sem1 wk10,NaN,NaN,Accelerated Natural Language Processing - Lecture
0,*Tutorial,10:00,10:50,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",Appleton Tower,AT_G.05,Accelerated Natural Language Processing
1,*Seminar - Online Live,12:10,13:00,Sem1 wk1-Sem1 wk11,NaN,NaN,Accelerated Natural Language Processing
2,*Tutorial,13:10,14:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing


To group our data according to course name, we can do the following:

In [34]:
# Group data by variable of interest
grouped_by_name = timetable_data.groupby("Course Name")
# Retrieve a given group by name
grouped_by_name.get_group("Accelerated Natural Language Processing")

,Type,Start,End,Weeks,Building,Room,Course Name
0,Computer Workshop - Online Live,14:10,15:00,"Sem1 wk2-Sem1 wk3, Sem1 wk7, Sem1 wk9",NaN,NaN,Accelerated Natural Language Processing
0,*Tutorial,10:00,10:50,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",Appleton Tower,AT_G.05,Accelerated Natural Language Processing
1,*Seminar - Online Live,12:10,13:00,Sem1 wk1-Sem1 wk11,NaN,NaN,Accelerated Natural Language Processing
2,*Tutorial,13:10,14:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing
3,*Tutorial,14:10,15:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing
4,*Tutorial,15:10,16:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing
5,Computer Workshop - Online Live,16:10,17:00,Sem1 wk1,NaN,NaN,Accelerated Natural Language Processing
6,*Tutorial,16:10,17:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing
0,*Tutorial,10:00,10:50,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",Appleton Tower,AT_G.05,Accelerated Natural Language Processing
1,*Tutorial,11:10,12:00,"Sem1 wk4-Sem1 wk5, Sem1 wk7-Sem1 wk10",lister-learning-and-teaching-centre,LLTC_3.1,Accelerated Natural Language Processing


Computations can then be performed on the grouped DataFrame like finding the size of each group:

In [35]:
# Count the number of 'activities' scheduled for each course in semester 1
grouped_by_name.size()

Course Name
Accelerated Natural Language Processing                    14
Accelerated Natural Language Processing - Lecture           1
Automated Reasoning                                         3
Bioinformatics 1                                            1
Bioinformatics 1 - Tutorial/Lab                             2
Blockchains and Distributed Ledgers                         1
Case Studies in Design Informatics 1                        8
Case Studies in Design Informatics 2                        3
Computational Cognitive Science                             6
Computer Architecture and Design                            9
Computer Graphics (Level 11)                                5
Computing in the Classroom                                  1
Data-driven Business and Behaviour                          8
Design and Analysis of Parallel Algorithms                  2
Discrete Mathematics and Probability                       18
Doing Research in Natural Language Processing             

We can check that a few of the groups have the size that we expect in the list above:

In [36]:
print(len(grouped_by_name.get_group("Accelerated Natural Language Processing")))
print(len(grouped_by_name.get_group("Natural Computing")))

14
16


We can also group by multiple variables:

In [37]:
# For each course, display the number of scheduled entries for each activity type
timetable_data.groupby(["Course Name", "Type"]).size()

Course Name                                        Type                           
Accelerated Natural Language Processing            *Seminar - Online Live              1
                                                   *Tutorial                          10
                                                   Computer Workshop - Online Live     3
Accelerated Natural Language Processing - Lecture  *Lecture - Online Live              1
Automated Reasoning                                *Lecture                            2
                                                                                      ..
Software Design and Modelling                      *Lecture - Online Live              1
Text Technologies for Data Science                 *Laboratory - Online Live           2
                                                   *Lecture - Online Live              1
Threaded Programming                               *Lecture                            2
                           

You can apply various functions to `GroupBy` objects. For example `.mean()` and `.median()` compute the mean and median of the numeric variables in each group.

**Exercise 10:** Ignoring entries which have NaN values, perform group-wise computations on the timetable data (and refer to pandas documentation) to answer the following questions:

a) Which building has the largest number of rooms in it? 

b) Which room is used the most across all the buildings, and which building is it in?

c) What is the mean and median length of a 'Lecture' in Informatics?

d) Which on average, is the longest activity type scheduled (i.e. lectures, tutorials, labs, etc.)?

In [49]:
buildinggroup = timetable_data.groupby("Building")
buildingNames = timetable_data['Building'].unique()
# To remove the nan entry
buildingNames = buildingNames[1:]
for buildingName in buildingNames:
    print(buildingName + ":", buildinggroup.get_group(buildingName)['Room'].count())    

Appleton Tower: 175
lister-learning-and-teaching-centre: 17
40 George Square Lower Teaching Hub: 38
7 George Square: 1
40 George Square: 4
40 George Square Lecture Theatres: 16
Old College: 4
Medical School: 1
Bayes Centre: 9
George Square Theatre: 1
Informatics Forum: 4


In [61]:
roomgroups = timetable_data.groupby("Room")
roomNames = timetable_data["Room"].unique()[1:]
roomUses = []
for roomName in roomNames:
    roomUses.append(len(roomgroups.get_group(roomName)))
print(roomNames[np.argmax(roomUses)])

40GS_LG.07 - Teaching Studio


In [ ]:
# Your code

In [ ]:
# Your code